## Setup

In [1]:
# imports
import pandas as pd
import glob
import os

# geocoding imports
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import box

#### Function for trailing spaces

In [2]:
# strip trailing spaces for columns with string dtype
def strip_trailing_spaces(df):

    # loop through columns
    for col in df.columns:
        # if string column then strip
        if df[col].dtype == 'object':
            df[col] = df[col].str.strip()

        # return df
        return df

#### Get citbike df

In [3]:
# find the csv files
files_list = glob.glob(os.path.join('Resources', '*.csv'))

# combine them
dfs = []

for file in files_list:
    df = pd.read_csv(file)
    dfs.append(df)

citibike_df = pd.concat(dfs, ignore_index=True)

# display df
citibike_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,0744109F13385D1D,electric_bike,2024-01-15 15:18:07,2024-01-15 15:32:44,Morris Canal,JC072,Oakland Ave,JC022,40.712297,-74.038185,40.737604,-74.052478,member
1,B1488BFEF9118000,classic_bike,2024-01-13 15:32:50,2024-01-13 15:36:18,JC Medical Center,JC110,Grove St PATH,JC115,40.715391,-74.049692,40.719410,-74.043090,member
2,95A2FE8E51B4C836,classic_bike,2024-01-19 13:11:00,2024-01-19 13:14:44,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,40.716366,-74.034344,member
3,95D9AFF6A1652DC1,classic_bike,2024-01-23 07:03:49,2024-01-23 07:07:11,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,40.716366,-74.034344,member
4,5F7408988A83B1B3,classic_bike,2024-01-01 16:46:10,2024-01-01 16:50:31,Morris Canal,JC072,Harborside,JC104,40.712419,-74.038526,40.719252,-74.034234,member


#### Get Stations df

In [4]:
# get the stations df
station_path = 'Output/stations.csv'
stations_df = pd.read_csv(station_path)

# strip leading/trailing spaces
stations_df = strip_trailing_spaces(stations_df)

# display
stations_df.head()

,short_name,name,region_id,lat,lon
0,7082.08,23 Ave & 31 St,71.0,40.774233,-73.912749
1,3460.06,2 Ave & 37 St,71.0,40.656240,-74.009330
2,8472.06,Valentine Ave & E 183 St,71.0,40.856987,-73.898237
3,5540.06,56 Dr & 61 St,71.0,40.723680,-73.904580
4,8717.07,E 201 St & Briggs Ave,71.0,40.872070,-73.884590


#### Get States df

In [5]:
# get states df
states_path = 'Resources/us_states_info/us-state-ansi-fips.csv'
states_fips_df = pd.read_csv(states_path, dtype={' st': str}).rename(columns={
    ' st': 'st_fip',
    ' stusps': 'st_abbr'
})

# strip leading/trailing spaces
states_fips_df = strip_trailing_spaces(states_fips_df)

# display
states_fips_df.head()

,stname,st_fip,st_abbr
0,Alabama,01,AL
1,Alaska,02,AK
2,Arizona,04,AZ
3,Arkansas,05,AR
4,California,06,CA


## Data Check
---

### Update data types

In [6]:
# check data types
citibike_df.dtypes

ride_id                object
rideable_type          object
started_at             object
ended_at               object
start_station_name     object
start_station_id       object
end_station_name       object
end_station_id         object
start_lat             float64
start_lng             float64
end_lat               float64
end_lng               float64
member_casual          object
dtype: object

In [7]:
# check for amount of nulls 
citibike_df.isnull().sum()

ride_id                  0
rideable_type            0
started_at               0
ended_at                 0
start_station_name     104
start_station_id       104
end_station_name      3061
end_station_id        3338
start_lat                0
start_lng                0
end_lat                353
end_lng                353
member_casual            0
dtype: int64

In [8]:
# update dates to datetime
date_format='mixed'
citibike_df['started_at'] = pd.to_datetime(citibike_df['started_at'], format=date_format)
citibike_df['ended_at'] = pd.to_datetime(citibike_df['ended_at'], format=date_format)

In [9]:
# check for datetime
citibike_df.select_dtypes('datetime64[ns]')

,started_at,ended_at
0,2024-01-15 15:18:07.000,2024-01-15 15:32:44.000
1,2024-01-13 15:32:50.000,2024-01-13 15:36:18.000
2,2024-01-19 13:11:00.000,2024-01-19 13:14:44.000
3,2024-01-23 07:03:49.000,2024-01-23 07:07:11.000
4,2024-01-01 16:46:10.000,2024-01-01 16:50:31.000
...,...,...
1052446,2024-12-28 09:45:30.704,2024-12-28 09:48:02.706
1052447,2024-12-12 16:21:50.427,2024-12-12 16:26:34.069
1052448,2024-12-11 19:23:24.109,2024-12-11 19:25:07.612
1052449,2024-12-12 20:48:40.471,2024-12-12 20:52:41.722


### Strip trailing spaces

In [10]:
# strip trailing spaces
citibike_df = strip_trailing_spaces(citibike_df)

### Check unique values

In [11]:
citibike_df.shape

(1052451, 13)

In [12]:
# check number of unique values per column
citibike_df.nunique()

ride_id               1052427
rideable_type               2
started_at            1043629
ended_at              1044175
start_station_name        205
start_station_id          205
end_station_name          519
end_station_id            520
start_lat              104008
start_lng              110731
end_lat                   541
end_lng                   547
member_casual               2
dtype: int64

##### Look into why Ride Id isn't unique --> counting rides from one month to the next (unnecessary duplicates)

In [13]:
# get df of duplicated ride_ids
dupes_df = citibike_df[citibike_df.duplicated(subset=['ride_id'], keep=False)]

# sort so duplicate groups stay together
dupes_df = dupes_df.sort_values(by='ride_id')

# display
dupes_df

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
420821,09D67B6866E802DA,classic_bike,2024-05-31 23:54:11.503,2024-06-01 00:19:31.421,Willow Ave & 12 St,HB505,Willow Ave & 12 St,HB505,40.751867,-74.030377,40.751867,-74.030377,casual
342531,09D67B6866E802DA,classic_bike,2024-05-31 23:54:11.000,2024-06-01 00:19:31.000,Willow Ave & 12 St,HB505,Willow Ave & 12 St,HB505,40.751867,-74.030377,40.751867,-74.030377,casual
360447,15413FAB9CC9F156,electric_bike,2024-05-31 23:51:10.638,2024-06-01 00:02:52.559,Hilltop,JC019,York St & Marin Blvd,JC097,40.731119,-74.057494,40.716615,-74.042412,casual
283209,15413FAB9CC9F156,electric_bike,2024-05-31 23:51:10.000,2024-06-01 00:02:52.000,Hilltop,JC019,York St & Marin Blvd,JC097,40.731119,-74.057494,40.716615,-74.042412,casual
256909,1AB4E466DCCC4147,classic_bike,2024-05-31 23:38:55.000,2024-06-01 05:20:10.000,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,Hoboken Ave at Monmouth St,JC105,40.736982,-74.027781,40.735208,-74.046964,casual
414293,1AB4E466DCCC4147,classic_bike,2024-05-31 23:38:55.590,2024-06-01 05:20:10.327,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,Hoboken Ave at Monmouth St,JC105,40.736982,-74.027781,40.735208,-74.046964,casual
363069,269CAE1A61663378,classic_bike,2024-05-31 23:55:31.802,2024-06-01 00:00:50.426,Hoboken Terminal - River St & Hudson Pl,HB102,Madison St & 1 St,HB402,40.736068,-74.029127,40.738790,-74.039300,member
273924,269CAE1A61663378,classic_bike,2024-05-31 23:55:31.000,2024-06-01 00:00:50.000,Hoboken Terminal - River St & Hudson Pl,HB102,Madison St & 1 St,HB402,40.736068,-74.029127,40.738790,-74.039300,member
395980,2FBED42B501D1159,electric_bike,2024-05-31 23:57:51.452,2024-06-01 00:08:06.554,Adams St & 12 St,HB610,City Hall - Washington St & 1 St,HB105,40.751833,-74.033343,40.737360,-74.030970,casual
305232,2FBED42B501D1159,electric_bike,2024-05-31 23:57:51.000,2024-06-01 00:08:06.000,Adams St & 12 St,HB610,City Hall - Washington St & 1 St,HB105,40.751833,-74.033343,40.737360,-74.030970,casual


In [14]:
# drop duplicate rows
citibike_df = citibike_df.drop_duplicates(subset=['ride_id'], keep='first')

# show number of unique ride id rows versus count
print(f'Total: {citibike_df['ride_id'].count()}')
print(f'Unique: {citibike_df['ride_id'].nunique()}')

Total: 1052427
Unique: 1052427


### Check nulls

In [15]:
# list total nulls
citibike_df.isna().sum()

ride_id                  0
rideable_type            0
started_at               0
ended_at                 0
start_station_name     104
start_station_id       104
end_station_name      3057
end_station_id        3334
start_lat                0
start_lng                0
end_lat                352
end_lng                352
member_casual            0
dtype: int64

## Fill Station Info Nulls/Update Broad Lat/Lngs
---

### Check # of rows affected by rounded lat/lngs (2 decimal places)

In [16]:
# def to split df into three based on if both lat/lng have two decimal places
def split_df(df, start_lat, start_lng, end_lat, end_lng):

    # masks to split
    start_mask = (
        (df[start_lat].apply(lambda x: f'{x:.2f}' == str(x)))
        &
        (df[start_lng].apply(lambda x: f'{x:.2f}' == str(x)))
    )
    end_mask = (
        (df[end_lat].apply(lambda x: f'{x:.2f}' == str(x)))
        &
        (df[end_lng].apply(lambda x: f'{x:.2f}' == str(x)))
    )

    # split dfs based on if lat/lngs are rounded by 2
    start_rounded__df = df[start_mask].copy()
    end_rounded_df = df[end_mask].copy()
    non_rounded_df = df[~start_mask & ~end_mask].copy()

    # return
    return (
        start_rounded__df.reset_index(drop=True), 
        end_rounded_df.reset_index(drop=True), 
        non_rounded_df.reset_index(drop=True)
    )

In [17]:
# split data
start_rounded_df, end_rounded_df, non_rounded_df = split_df(citibike_df.copy(), 'start_lat', 'start_lng', 'end_lat', 'end_lng')

# check # of rows for outputs
print(f'Start coordinates (rounded to 2 decimals places): {len(start_rounded_df)}')
print(f'End coordinates (rounded to 2 decimals places): {len(end_rounded_df)}')
print(f'Neither start or end coordinates are rounded: {len(non_rounded_df)}')

Start coordinates (rounded to 2 decimals places): 104
End coordinates (rounded to 2 decimals places): 3033
Neither start or end coordinates are rounded: 1049336


### Correlation between rounded lat/lng and null station info

In [18]:
# show nulls on split data
rounded_dfs = [start_rounded_df, end_rounded_df, non_rounded_df]
[{'Rows': len(df), 
  'Nulls': df[['start_station_name', 'start_station_id', 'start_lat', 'end_station_name', 'end_station_id', 'end_lat']].isna().sum()} 
  for df in rounded_dfs]

[{'Rows': 104,
  'Nulls': start_station_name    104
  start_station_id      104
  start_lat               0
  end_station_name       46
  end_station_id         46
  end_lat                 0
  dtype: int64},
 {'Rows': 3033,
  'Nulls': start_station_name      46
  start_station_id        46
  start_lat                0
  end_station_name      2756
  end_station_id        3033
  end_lat                352
  dtype: int64},
 {'Rows': 1049336,
  'Nulls': start_station_name      0
  start_station_id        0
  start_lat               0
  end_station_name      301
  end_station_id        301
  end_lat                 0
  dtype: int64}]

### Get unique rounded stations df

In [19]:
# see how many dupes from rounded lat/lng combos
# loop through
round_nums = list(range(1, 6))
dupes_list = []

for num in round_nums:
    # see if rounded lat/lngs are unique in station
    rounded_stations = stations_df.copy()
    rounded_stations.lat = round(rounded_stations.lat, num)
    rounded_stations.lon = round(rounded_stations.lon, num)

    # groupby lat/lng and count
    group_rounded = rounded_stations.groupby(['lat', 'lon']).size().reset_index(name='count')
    
    # output
    dupe_cnt = len(group_rounded[group_rounded['count'] > 1])
    dupes_list.append({
        'Rounded by': num, 
        'Dupe Count': dupe_cnt,
        'Dupe %': f'{round((dupe_cnt / len(group_rounded))*100, 4)}%'
    })

# display
dupes_list

[{'Rounded by': 1, 'Dupe Count': 9, 'Dupe %': '90.0%'},
 {'Rounded by': 2, 'Dupe Count': 262, 'Dupe %': '91.2892%'},
 {'Rounded by': 3, 'Dupe Count': 10, 'Dupe %': '0.4502%'},
 {'Rounded by': 4, 'Dupe Count': 0, 'Dupe %': '0.0%'},
 {'Rounded by': 5, 'Dupe Count': 0, 'Dupe %': '0.0%'}]

In [20]:
# save unique lat/lng rounded by 2 df
rounded_stations_df = stations_df.copy()
rounded_stations_df[['rounded_lat', 'rounded_lon']] = round(rounded_stations_df[['lat', 'lon']], 2)

# group by
group_r2_stations_df = rounded_stations_df.groupby(['rounded_lat', 'rounded_lon']).size().reset_index(name='count')

# get unique lat/lng combos
unique_r2_coords_df = group_r2_stations_df[group_r2_stations_df['count']==1].copy()

# merge to filter for only rows that match these unique combos
unique_rounded_stations_df = pd.merge(
    rounded_stations_df,
    unique_r2_coords_df[['rounded_lat', 'rounded_lon']],
    on=['rounded_lat', 'rounded_lon'],
    how='inner'
)

# display
print(f'Number of rows: {len(unique_rounded_stations_df)}')
unique_rounded_stations_df.head()

Number of rows: 25


,short_name,name,region_id,lat,lon,rounded_lat,rounded_lon
0,5579.04,55 Dr & Maurice Ave,71.0,40.726100,-73.909170,40.73,-73.91
1,5388.01,Kent Ave & Grand St,71.0,40.716425,-73.965940,40.72,-73.97
2,6869.06,Butler St & 27 Ave,71.0,40.765210,-73.864830,40.77,-73.86
3,3220.01,58 St & Pier 4,71.0,40.646475,-74.026081,40.65,-74.03
4,7820.05,Hunts Point Ave & Bryant Ave,71.0,40.813991,-73.885500,40.81,-73.89


### Function to fill null station info and update rounded lat/lngs

In [21]:
def fix_rounded_side(rides_df, stations_df, side='start'):
    '''
    Two merges (based on 'start' or 'end' coordinates):
    1) station name--> fill station id & lat/lng
    2) lat/lng (rounded)--> fill station id & lat/lng

    Then put new columns into main df

    returns copy of rides_df with updated columns for the specified side
    '''

    # work on a copy
    df = rides_df.copy()

    # column names based on start or end
    station_id = side + '_station_id'
    station_name = side + '_station_name'
    lat = side + '_lat'
    lng = side + '_lng'

    # ----------------------------------------
    # MERGE ON STATION NAME --> Get station ID
    # ----------------------------------------
    # filter to rows missing station_id but have station name
    null_ids_df = df[df[station_id].isna() & df[station_name].notna()].copy()

    # lower station names before merging
    null_ids_df[station_name] = null_ids_df[station_name].str.lower().str.strip()
    nlower_stations_df = stations_df.copy()
    nlower_stations_df['name'] = nlower_stations_df['name'].str.lower().str.strip()

    # merge on station name
    not_null_ids_df = pd.merge(
        null_ids_df,
        nlower_stations_df[['short_name', 'name', 'lat', 'lon']],
        left_on=station_name,
        right_on='name',
        how='left'
    )

    # merge updates back into main df based on ride_id
    df = df.merge(
        not_null_ids_df[['ride_id', 'short_name', 'lat', 'lon']],
        on='ride_id',
        how='left'
    )

    # coalesce: if station match found, use it
    df[lat] = df['lat'].combine_first(df[lat])
    df[lng] = df['lon'].combine_first(df[lng])
    df[station_id] = df['short_name'].combine_first(df[station_id])

    # drop old columns
    df = df.drop(['short_name', 'lat', 'lon'], axis=1)

    # ---------------------------------------------
    # MERGE ON ROUNDED LAT/LNG --> Get station info
    # ---------------------------------------------
    # filter to rows missing station ids but have lat/lng
    station_nulls_df = df[df[station_name].isna() & df[lat].notna() & df[lng].notna()].copy()

    # make sure they're rounded lat/lng values (2 decimals)
    rounded_nulls_df = station_nulls_df[
        (station_nulls_df[lat].apply(lambda x: f'{x:.2f}' == str(x)))
        &
        (station_nulls_df[lng].apply(lambda x: f'{x:.2f}' == str(x)))
    ]

    # see if they match up with unique rounded by 2 lat/lon combos for stations
    rounded_nulls_df = pd.merge(
        rounded_nulls_df,
        unique_rounded_stations_df,
        left_on=[lat, lng],
        right_on=['rounded_lat', 'rounded_lon'],
        how='left'
    )

    # merge with main df
    df = df.merge(
        rounded_nulls_df[['ride_id', 'short_name', 'name', 'lat', 'lon']],
        on='ride_id',
        how='left'
    )

    # coalesce: if station match found, use it
    df[lat] = df['lat'].combine_first(df[lat])
    df[lng] = df['lon'].combine_first(df[lng])
    df[station_id] = df['short_name'].combine_first(df[station_id])
    df[station_name] = df['name'].combine_first(df[station_name])

    # drop old columns
    df = df.drop(['short_name', 'name', 'lat', 'lon'], axis=1)

    # return
    return df


In [22]:
# display nulls before
print(f'Number of rows: {len(citibike_df)}')
citibike_df[['start_station_name', 'start_station_id', 'end_station_name', 'end_station_id']].isna().sum()

Number of rows: 1052427


start_station_name     104
start_station_id       104
end_station_name      3057
end_station_id        3334
dtype: int64

### Update Start Lat/Lng

In [23]:
# fix start side
fixed_citibike_df = fix_rounded_side(citibike_df, stations_df, side='start')

# display nulls after
print(f'Number of rows: {len(fixed_citibike_df)}')
fixed_citibike_df[['start_station_name', 'start_station_id', 'end_station_name', 'end_station_id']].isna().sum()

Number of rows: 1052427


start_station_name      97
start_station_id        97
end_station_name      3057
end_station_id        3334
dtype: int64

### Update End Lat/Lng

In [24]:
# fix end side
fixed_citibike_df = fix_rounded_side(fixed_citibike_df, stations_df, side='end')

# display nulls after
print(f'Number of rows: {len(fixed_citibike_df)}')
fixed_citibike_df[['start_station_name', 'start_station_id', 'end_station_name', 'end_station_id']].isna().sum()

Number of rows: 1052427


start_station_name      97
start_station_id        97
end_station_name      2744
end_station_id        2744
dtype: int64

## Reverse Geocoding
---

### Get GeoDataFrames

#### Bounding Boxes

In [25]:
# roughly cover areas of interest
bbox_ny = box(-79.7624, 40.4961, -71.8562, 45.0153)  #for New York
bbox_nyc = box(-74.3, 40.4, -73.6, 41.0)  #bounding box for NYC area
bbox_states = box(
    -80.52,  #min longitude (~ southwestern PA)
    38.45,   #min latitude  (~ southern Delaware)
    -71.85,  #max longitude (eastern Long Island, NY)
    45.02    #max latitude  (~ upstate NY near Canada)
)  #roughly cover DE, NJ, NY, and PA

#### CitiBike gdf

In [26]:
# create geometry columsn of point objects from lat/lng
fixed_citibike_df['geometry_start'] = gpd.points_from_xy(fixed_citibike_df['start_lng'], fixed_citibike_df['start_lat'])
fixed_citibike_df['geometry_end'] = gpd.points_from_xy(fixed_citibike_df['end_lng'], fixed_citibike_df['end_lat'])


# construct gdf
citibike_gdf = gpd.GeoDataFrame(
    fixed_citibike_df,
    geometry='geometry_start',  #current active geometry
    crs='EPSG:4326'
)

# display
citibike_gdf.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,geometry_start,geometry_end
0,0744109F13385D1D,electric_bike,2024-01-15 15:18:07,2024-01-15 15:32:44,Morris Canal,JC072,Oakland Ave,JC022,40.712297,-74.038185,40.737604,-74.052478,member,POINT (-74.038 40.712),POINT (-74.052 40.738)
1,B1488BFEF9118000,classic_bike,2024-01-13 15:32:50,2024-01-13 15:36:18,JC Medical Center,JC110,Grove St PATH,JC115,40.715391,-74.049692,40.719410,-74.043090,member,POINT (-74.05 40.715),POINT (-74.043 40.719)
2,95A2FE8E51B4C836,classic_bike,2024-01-19 13:11:00,2024-01-19 13:14:44,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,40.716366,-74.034344,member,POINT (-74.039 40.712),POINT (-74.034 40.716)
3,95D9AFF6A1652DC1,classic_bike,2024-01-23 07:03:49,2024-01-23 07:07:11,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,40.716366,-74.034344,member,POINT (-74.039 40.712),POINT (-74.034 40.716)
4,5F7408988A83B1B3,classic_bike,2024-01-01 16:46:10,2024-01-01 16:50:31,Morris Canal,JC072,Harborside,JC104,40.712419,-74.038526,40.719252,-74.034234,member,POINT (-74.039 40.712),POINT (-74.034 40.719)


#### Cities, Counties, States gdf

In [27]:
# find county subdivisions shapefiles
cousub_shp_list = glob.glob(os.path.join('Resources/Shapefiles_Cousub/Cousub_*/*.shp'))

# read in each county subdivision file into list
list_cousub_gdf = []

for folder in cousub_shp_list:
    cousub_gdf = gpd.read_file(folder).to_crs('EPSG:4326')
    list_cousub_gdf.append(cousub_gdf)

# combine gdfs into one gdf
cousubs_gdf = pd.concat(list_cousub_gdf, ignore_index=True)

# display
cousubs_gdf.head(3)

,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,GEOIDFQ,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,PLACEFP,PLACENS,PCICBSA
0,10,005,91776,01935613,1000591776,0600000US1000591776,Lewes,Lewes CCD,22,Z5,G4040,S,195797331,235998103,+38.7368667,-075.1201635,"POLYGON ((-75.309 38.824, -75.309 38.824, -75....",NaN,NaN,NaN
1,10,005,92516,01935618,1000592516,0600000US1000592516,Millsboro,Millsboro CCD,22,Z5,G4040,S,235822742,21674892,+38.6121774,-075.2861957,"POLYGON ((-75.484 38.62, -75.482 38.622, -75.4...",NaN,NaN,NaN
2,10,001,90444,01935604,1000190444,0600000US1000190444,Central Kent,Central Kent CCD,22,Z5,G4040,S,213522622,780887,+39.0590474,-075.6000618,"POLYGON ((-75.742 39.08, -75.742 39.081, -75.7...",NaN,NaN,NaN


In [28]:
# read in county shapefile (for NYC counties)
counties_shapefile = 'Resources/Shapefile_Counties'

counties_gdf = gpd.read_file(
    counties_shapefile,
    #bbox=bbox_nyc
    bbox=bbox_ny
).to_crs('EPSG:4326')

# look at geodataframe
counties_gdf.head(3)

,STATEFP,COUNTYFP,COUNTYNS,GEOID,GEOIDFQ,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,36,101,00974148,36101,0500000US36101,Steuben,Steuben County,06,H1,G4020,236,18500,None,A,3601403378,35034921,+42.2667252,-077.3855253,"POLYGON ((-77.083 42, -77.094 42, -77.096 42, ..."
1,42,053,01213669,42053,0500000US42053,Forest,Forest County,06,H1,G4020,None,None,None,A,1106595595,8369676,+41.5133040,-079.2497050,"POLYGON ((-78.959 41.525, -78.959 41.525, -78...."
2,25,015,00606934,25015,0500000US25015,Hampshire,Hampshire County,06,H4,G4020,521,11200,None,N,1365533874,46625894,+42.3394593,-072.6636936,"POLYGON ((-72.336 42.22, -72.336 42.22, -72.33..."


#### zipcodes gdf

In [29]:
# read zip code shapefile
zip_shapefile = 'Resources/Shapefile_ZipCode'
zip_gdf = gpd.read_file(
    zip_shapefile,
    bbox=bbox_states
).to_crs('EPSG:4326')

# look at geodataframe
zip_gdf.head()

,ZCTA5CE20,GEOID20,GEOIDFQ20,CLASSFP20,MTFCC20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry
0,20851,20851,860Z200US20851,B5,G6350,S,6568841,662,+39.0780702,-077.1208381,"POLYGON ((-77.141 39.08, -77.139 39.08, -77.13..."
1,21776,21776,860Z200US21776,B5,G6350,S,91813880,302083,+39.5180058,-077.0997095,"POLYGON ((-77.171 39.477, -77.171 39.478, -77...."
2,21252,21252,860Z200US21252,B5,G6350,S,643789,0,+39.3941158,-076.6113826,"POLYGON ((-76.622 39.396, -76.622 39.396, -76...."
3,20623,20623,860Z200US20623,B5,G6350,S,4969047,19273,+38.7416148,-076.8424560,"POLYGON ((-76.86 38.736, -76.86 38.737, -76.85..."
4,21798,21798,860Z200US21798,B5,G6350,S,42212361,118738,+39.5377505,-077.2950783,"POLYGON ((-77.359 39.557, -77.359 39.558, -77...."


### Get Zipcodes

#### Add exact zipcodes

In [30]:
# get zipcodes reverse geocoding function
def get_zipcodes(main_gdf, zipcodes_gdf, side='start'):

    # column name based on start or end
    zip = side + '_zip'
    geometry_col = 'geometry_' + side

    # work on copy
    rides_gdf = main_gdf.copy()

    # if side=end then set geometry
    rides_gdf = gpd.GeoDataFrame(
            rides_gdf,
            geometry=geometry_col,
            crs='EPSG:4326'
        )

    # join with zipcode gdf
    zips_joined_gdf = gpd.sjoin(rides_gdf, zipcodes_gdf, how='left', predicate='within').rename(columns={'ZCTA5CE20': zip})

    # merge zip col back into citibike gdf
    rides_gdf[zip] = zips_joined_gdf[zip]

    # return
    return(rides_gdf)

In [31]:
# exact match zipcodes
start_z_citibike_gdf = get_zipcodes(citibike_gdf, zip_gdf, side='start')
citibike_gdf = get_zipcodes(start_z_citibike_gdf, zip_gdf, side='end')

# list null totals
citibike_gdf[['start_lat', 'start_lng', 'end_lat', 'end_lng', 'start_zip', 'end_zip']].isna().sum()

start_lat      0
start_lng      0
end_lat      352
end_lng      352
start_zip      0
end_zip      398
dtype: int64

#### Add nearest zipcodes for missing end zips (right outside of boundaries such as on water)

In [32]:
# get NEAREST zipcode reverse geocoding function (do it if end_zip nulls > end lat/lng nulls)
def get_nearest_zipcodes(main_gdf, zipcodes_gdf, side='end'):

    # column name based on start or end
    zip = side + '_zip'
    lat = side + '_lat'
    lng = side + '_lng'
    nearest_zip = 'nearest_' + zip
    geometry_col = 'geometry_' + side

    # work on copy
    rides_gdf = main_gdf.copy()

    # filter to null zips with coordinates
    zip_nulls_gdf = rides_gdf[
        rides_gdf[zip].isna() &
        rides_gdf[lat].notna() &
        rides_gdf[lng].notna()
    ].copy()

    # fix gdf setup of zip nulls with coordinates
    zip_nulls_gdf = zip_nulls_gdf.iloc[:, :-1].reset_index(drop=True)

    # set geometry to side coordinates and crs
    zip_nulls_gdf = gpd.GeoDataFrame(
            zip_nulls_gdf,
            geometry=geometry_col,
            crs='EPSG:4326'
        )

    # reproject points and zip polygons to local projected crs
    points_proj = zip_nulls_gdf.to_crs('EPSG:2263')
    zip_proj = zipcodes_gdf.to_crs('EPSG:2263')

    # nearest spatial join to get zips
    zip_nearest_gdf = gpd.sjoin_nearest(
        points_proj, zip_proj,
        how='left',
        distance_col=f'dist_to_{zip}'
    )

    # rename zip column
    zip_nearest_gdf = zip_nearest_gdf.rename(columns={'ZCTA5CE20': nearest_zip})

    # merge end nearest zips column to main gdf
    rides_gdf = rides_gdf.merge(
        zip_nearest_gdf[['ride_id', nearest_zip]],
        on='ride_id',
        how='left'
    )

    # fill in missing zip col values with nearest zips
    rides_gdf[zip] = rides_gdf[zip].fillna(rides_gdf[nearest_zip])

    # drop nearest zip column
    rides_gdf = rides_gdf.drop(nearest_zip, axis=1)

    # return
    return(rides_gdf)

In [33]:
# nearest match zipcodes
citibike_gdf = get_nearest_zipcodes(citibike_gdf, zip_gdf, side='end')

# list null totals (null end coordinates should = null end zips)
citibike_gdf[['start_lat', 'start_lng', 'end_lat', 'end_lng', 'start_zip', 'end_zip']].isna().sum()

start_lat      0
start_lng      0
end_lat      352
end_lng      352
start_zip      0
end_zip      352
dtype: int64

In [34]:
# display updated gdf
citibike_gdf.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,geometry_start,geometry_end,start_zip,end_zip
0,0744109F13385D1D,electric_bike,2024-01-15 15:18:07,2024-01-15 15:32:44,Morris Canal,JC072,Oakland Ave,JC022,40.712297,-74.038185,40.737604,-74.052478,member,POINT (-74.038 40.712),POINT (-74.052 40.738),07302,07306
1,B1488BFEF9118000,classic_bike,2024-01-13 15:32:50,2024-01-13 15:36:18,JC Medical Center,JC110,Grove St PATH,JC115,40.715391,-74.049692,40.719410,-74.043090,member,POINT (-74.05 40.715),POINT (-74.043 40.719),07302,07302
2,95A2FE8E51B4C836,classic_bike,2024-01-19 13:11:00,2024-01-19 13:14:44,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,40.716366,-74.034344,member,POINT (-74.039 40.712),POINT (-74.034 40.716),07302,07302
3,95D9AFF6A1652DC1,classic_bike,2024-01-23 07:03:49,2024-01-23 07:07:11,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,40.716366,-74.034344,member,POINT (-74.039 40.712),POINT (-74.034 40.716),07302,07302
4,5F7408988A83B1B3,classic_bike,2024-01-01 16:46:10,2024-01-01 16:50:31,Morris Canal,JC072,Harborside,JC104,40.712419,-74.038526,40.719252,-74.034234,member,POINT (-74.039 40.712),POINT (-74.034 40.719),07302,07311


### Get Cities, Counties, States

In [35]:
# get cities, counties, states reverse geocoding function
def get_cities_counties_states(main_gdf, cousub_gdf, side='start'):

    # column names based on start or end
    city = side + '_city'
    county = side + '_county'
    state = side + '_state'
    geometry_col = 'geometry_' + side

    # work on copy
    rides_gdf = main_gdf.copy()

    # set geometry accordingly
    rides_gdf = gpd.GeoDataFrame(
            rides_gdf.copy(),
            geometry=geometry_col,
            crs='EPSG:4326'
        )

    # set predicate accordingly
    if side == 'end':
        predicate_option = 'intersects'
    else:
        predicate_option = 'within'

    # join with cities
    cities_joined_gdf = gpd.sjoin(rides_gdf, cousub_gdf, how='left', predicate=predicate_option).rename(columns={
        'NAME': city,
        'COUNTYFP': county,
        'STATEFP': state
    })

    # merge back into citibike gdf by ride id
    rides_gdf = rides_gdf.merge(
        cities_joined_gdf[['ride_id', city, county, state]],
        on='ride_id',
        how='left'
    )

    # return 
    return(rides_gdf)

In [36]:
# get cities, counties, and states
starts_citibike_gdf = get_cities_counties_states(citibike_gdf, cousubs_gdf, side='start')
citibike_gdf = get_cities_counties_states(starts_citibike_gdf, cousubs_gdf, side='end')

# list null totals
citibike_gdf[['start_lat', 'start_lng', 'end_lat', 'end_lng', 
              'start_city', 'start_county', 'start_state',
              'end_city', 'end_county', 'end_state']].isna().sum()

start_lat          0
start_lng          0
end_lat          352
end_lng          352
start_city         0
start_county       0
start_state        0
end_city         352
end_county      3342
end_state        352
dtype: int64

#### Fill in null end counties (from NY)

In [37]:
# def to fill in null counties (mainly from NY) --> END
def get_missing_counties(main_gdf, counties_gdf, side='end'):

    # column names based on start or end
    city = side + '_city'
    county = side + '_county'
    state = side + '_state'
    geometry_col = 'geometry_' + side
    new_county = 'new_' + county

    # work on copy
    rides_gdf = main_gdf.copy()

    # gdf for missing counties
    null_counties_with_cities = rides_gdf[rides_gdf[county].isna() & rides_gdf[city].notna()].copy()

    # drop columns to fix
    counties_nulls_gdf = null_counties_with_cities.drop([city, county, state], axis=1).reset_index(drop=True)

    # set geometry to side coordinates and crs
    counties_nulls_gdf = gpd.GeoDataFrame(
        counties_nulls_gdf,
        geometry=geometry_col,
        crs='EPSG:4326'
    )

    # create new counties column
    counties_nulls_gdf = gpd.sjoin(counties_nulls_gdf, counties_gdf, how='left', predicate='within').rename(columns={
        'COUNTYFP': new_county
    })

    # merge new end counties column to main gdf
    rides_gdf = rides_gdf.merge(
        counties_nulls_gdf[['ride_id', new_county]],
        on='ride_id',
        how='left'
    )

    # fill in missing county values with new county
    rides_gdf[county] = rides_gdf[county].fillna(rides_gdf[new_county])

    # drop new county column
    rides_gdf = rides_gdf.drop(new_county, axis=1)

    # return
    return(rides_gdf)

In [38]:
# get missing counties
citibike_gdf = get_missing_counties(citibike_gdf, counties_gdf, side='end')

# list null totals
citibike_gdf[['start_lat', 'start_lng', 'end_lat', 'end_lng', 
              'start_city', 'start_county', 'start_state',
              'end_city', 'end_county', 'end_state']].isna().sum()

start_lat         0
start_lng         0
end_lat         352
end_lng         352
start_city        0
start_county      0
start_state       0
end_city        352
end_county      352
end_state       352
dtype: int64

In [39]:
# look at updated gdf
citibike_gdf.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,geometry_start,geometry_end,start_zip,end_zip,start_city,start_county,start_state,end_city,end_county,end_state
0,0744109F13385D1D,electric_bike,2024-01-15 15:18:07,2024-01-15 15:32:44,Morris Canal,JC072,Oakland Ave,JC022,40.712297,-74.038185,...,POINT (-74.038 40.712),POINT (-74.052 40.738),07302,07306,Jersey City,017,34,Jersey City,017,34
1,B1488BFEF9118000,classic_bike,2024-01-13 15:32:50,2024-01-13 15:36:18,JC Medical Center,JC110,Grove St PATH,JC115,40.715391,-74.049692,...,POINT (-74.05 40.715),POINT (-74.043 40.719),07302,07302,Jersey City,017,34,Jersey City,017,34
2,95A2FE8E51B4C836,classic_bike,2024-01-19 13:11:00,2024-01-19 13:14:44,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,...,POINT (-74.039 40.712),POINT (-74.034 40.716),07302,07302,Jersey City,017,34,Jersey City,017,34
3,95D9AFF6A1652DC1,classic_bike,2024-01-23 07:03:49,2024-01-23 07:07:11,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,...,POINT (-74.039 40.712),POINT (-74.034 40.716),07302,07302,Jersey City,017,34,Jersey City,017,34
4,5F7408988A83B1B3,classic_bike,2024-01-01 16:46:10,2024-01-01 16:50:31,Morris Canal,JC072,Harborside,JC104,40.712419,-74.038526,...,POINT (-74.039 40.712),POINT (-74.034 40.719),07302,07311,Jersey City,017,34,Jersey City,017,34


## Last Data Check
---

### Remove geometry columns

In [40]:
# remove geometry columns
citibike_df = citibike_gdf.drop(['geometry_start', 'geometry_end'], axis=1)

# display shape and column names
print(citibike_df.shape)
citibike_df.columns

(1052427, 21)


Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'start_zip', 'end_zip', 'start_city', 'start_county',
       'start_state', 'end_city', 'end_county', 'end_state'],
      dtype='object')

In [41]:
# save col order
col_order = citibike_df.columns

### Update County fip to five digits (state fip + county fip)

In [42]:
# definition to create full five digit county fip (state fip + county fip)
def create_fullzip(main_df, side='start'):

    # column name based on side
    state = side + '_state'
    county = side + '_county'
    new_county = 'new_' + county

    # create copy
    rides_df = main_df.copy()

    # combine state fip & zip fip
    rides_df[new_county] = rides_df[state].astype(str) + rides_df[county].astype(str)

    # replace old zip column with full one
    rides_df = rides_df.drop(county, axis=1)  #drop old zip col
    rides_df[county] = rides_df[new_county]  #rename new zip col
    rides_df = rides_df.drop(new_county, axis=1)  #drop old new zip col

    # return
    return rides_df

In [43]:
# replace old county fip with five digit county fip
full_s_county_df = create_fullzip(citibike_df, side='start')
citibike_df = create_fullzip(full_s_county_df, side='end')

# display columns
citibike_df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'start_zip', 'end_zip', 'start_city', 'start_state',
       'end_city', 'end_state', 'start_county', 'end_county'],
      dtype='object')

### Replace State Fip with State Abbreviation

In [44]:
# function to replace state fip with state abbreviation
def match_state_abbr(main_df, state_df, side='start'):

    # column names
    state = side + '_state'

    # make copy of df
    rides_df = main_df.copy()

    # merge df with state fip
    rides_df = pd.merge(
        rides_df,
        state_df[['st_fip', 'st_abbr']],
        left_on=state,
        right_on='st_fip',
        how='left'
    )

    # replace state fip with state abbr
    rides_df = rides_df.drop(['st_fip', state], axis=1)  #drop fip cols
    rides_df[state] = rides_df['st_abbr'] #make state abbr the official state col
    rides_df = rides_df.drop('st_abbr', axis=1)  #remove old abbr col
    
    # return
    return rides_df
    

In [45]:
# match state fip with abbr
match_st_df = match_state_abbr(citibike_df, states_fips_df, side='start')
citibike_df = match_state_abbr(match_st_df, states_fips_df, side='end')

# display cols
print(citibike_df.columns)

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'start_zip', 'end_zip', 'start_city', 'end_city',
       'start_county', 'end_county', 'start_state', 'end_state'],
      dtype='object')


### Reorder columns

In [46]:
# reorder columns based on saved col order
citibike_df = citibike_df.reindex(columns=col_order)
print(citibike_df.columns)

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'start_zip', 'end_zip', 'start_city', 'start_county',
       'start_state', 'end_city', 'end_county', 'end_state'],
      dtype='object')


### Check nulls

In [47]:
# total nulls
citibike_df.isna().sum()

ride_id                     0
rideable_type               0
started_at                  0
ended_at                    0
start_station_name         97
start_station_id           97
end_station_name         2744
end_station_id           2744
start_lat                   0
start_lng                   0
end_lat                   352
end_lng                   352
member_casual               0
start_zip                   0
end_zip                   352
start_city                  0
start_county                0
start_state           1052427
end_city                  352
end_county                  0
end_state             1052427
dtype: int64

## Export
---

In [48]:
# display final df
print(f'Rows: {citibike_df.shape[0]}, Columns: {citibike_df.shape[1]}')
citibike_df.head()

Rows: 1052427, Columns: 21


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_zip,end_zip,start_city,start_county,start_state,end_city,end_county,end_state
0,0744109F13385D1D,electric_bike,2024-01-15 15:18:07,2024-01-15 15:32:44,Morris Canal,JC072,Oakland Ave,JC022,40.712297,-74.038185,...,-74.052478,member,07302,07306,Jersey City,34017,NaN,Jersey City,34017,NaN
1,B1488BFEF9118000,classic_bike,2024-01-13 15:32:50,2024-01-13 15:36:18,JC Medical Center,JC110,Grove St PATH,JC115,40.715391,-74.049692,...,-74.043090,member,07302,07302,Jersey City,34017,NaN,Jersey City,34017,NaN
2,95A2FE8E51B4C836,classic_bike,2024-01-19 13:11:00,2024-01-19 13:14:44,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,...,-74.034344,member,07302,07302,Jersey City,34017,NaN,Jersey City,34017,NaN
3,95D9AFF6A1652DC1,classic_bike,2024-01-23 07:03:49,2024-01-23 07:07:11,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,...,-74.034344,member,07302,07302,Jersey City,34017,NaN,Jersey City,34017,NaN
4,5F7408988A83B1B3,classic_bike,2024-01-01 16:46:10,2024-01-01 16:50:31,Morris Canal,JC072,Harborside,JC104,40.712419,-74.038526,...,-74.034234,member,07302,07311,Jersey City,34017,NaN,Jersey City,34017,NaN


In [49]:
# export output
citibike_df.to_csv('Output/new_cleaned_citibike_data.csv', index=False)

## Stations DF
---

### Data Check

In [50]:
# check number of unique values
print(f'Number of rows: {len(stations_df)}')
stations_df.nunique()

Number of rows: 2231


short_name    2231
name          2231
region_id        3
lat           2218
lon           2214
dtype: int64

### Check Station Info in CitiBike df

--------------

In [94]:
# function to group by station id and station name
def new_unique_stations_grp3(df, side='start'):
    # make copy
    main_df = df.copy()

    # col names
    station_id = f'{side}_station_id'
    station_name = f'{side}_station_name'

    # Step 1: unique pairs + ride counts
    pairs_df = (
        main_df
        .groupby([station_id, station_name], dropna=False)
        .size()
        .reset_index(name='ride_count')
    )

    # Step 2 & 3: compute unique counts
    unique_names_by_id = main_df.groupby(station_id)[station_name].nunique(dropna=False)
    unique_ids_by_name = main_df.groupby(station_name)[station_id].nunique(dropna=False)

    # Step 4: attach to pairs
    pairs_df['unique_names_for_id'] = pairs_df[station_id].map(unique_names_by_id)
    pairs_df['unique_ids_for_name'] = pairs_df[station_name].map(unique_ids_by_name)

    # check unique values
    print('---------------------------------------------------------------------')
    print(f'{side.upper()}')
    print('---------------------------------------------------------------------')
    print(f'Duplicated ids for name: {len(pairs_df[pairs_df['unique_names_for_id']>1])}')
    print(pairs_df[pairs_df['unique_names_for_id']>1][[station_id, station_name, 'unique_names_for_id']])
    print('---------------------------------------------------------------------')
    print(f'Duplicate names for id: {len(pairs_df[pairs_df['unique_ids_for_name']>1])}')
    print(f'{pairs_df[pairs_df['unique_ids_for_name']>1][[station_name, station_id, 'unique_ids_for_name']]}\n')
    
    # return df
    return pairs_df

In [95]:
# see duplicate ends
start_pairs_df = new_unique_stations_grp3(citibike_df, side='start')
end_pairs_df = new_unique_stations_grp3(citibike_df, side='end')

---------------------------------------------------------------------
START
---------------------------------------------------------------------
Duplicated ids for name: 0
Empty DataFrame
Columns: [start_station_id, start_station_name, unique_names_for_id]
Index: []
---------------------------------------------------------------------
Duplicate names for id: 0
Empty DataFrame
Columns: [start_station_name, start_station_id, unique_ids_for_name]
Index: []

---------------------------------------------------------------------
END
---------------------------------------------------------------------
Duplicated ids for name: 2
    end_station_id           end_station_name  unique_names_for_id
186        5772.05   Morton St & Greenwich St                  2.0
187        5772.05  Morton St & Washington St                  2.0
---------------------------------------------------------------------
Duplicate names for id: 4
       end_station_name end_station_id  unique_ids_for_name
214  E 17 St

----------------------

## MAIN CHANGES

## Update Stations Df with all Station Ids
---

### See if all station ids in CitiBike df are in official station info table

In [323]:
# function to find missing station ids
def get_missing_station_ids(df, stations_df):
    # make copy of df
    main_df = df.copy()
    station_df = stations_df.copy()

    # loop through both sides
    sides = ['start', 'end']
    missing_ids = []
    
    for side in sides:
        # get col names
        station_id = f'{side}_station_id'
        unique_ids = main_df[station_id].unique()
        
        # get values not found in stations_df
        unfound_values = [id for id in unique_ids
                          if id not in station_df['short_name'].values]
        missing_ids.append(unfound_values)

    # combine lists and get unique values
    unique_missing_ids = list(set(missing_ids[0] + missing_ids[1]))
    
    # return unique list
    return unique_missing_ids

In [324]:
# list missing station ids
missing_station_ids = get_missing_station_ids(citibike_df, stations_df)
missing_station_ids

['6920.03',
 '5065.04',
 '6801.01',
 '6762.04',
 '5980.07',
 '6432.10',
 '6839.10',
 '6779.04',
 '6098.12',
 '6303.01',
 '5470.02',
 'HB503',
 nan]

### Identify non-official station ids vs official

In [133]:
# function to group by station id and station name
def combine_station_info(df):
    # make copy
    main_df = df.copy()

    # loop through start and end sides
    sides = ['start', 'end']

    for side in sides:
        # col names
        station_id = f'{side}_station_id'
        station_name = f'{side}_station_name'

        # new col names
        new_col_names = {
            station_id: 'station_id',
            station_name: 'station_name'
        }

        # extract stations info into new df
        if side == 'start':
            start_stations_df = (main_df[[station_id, station_name]]
                                 .drop_duplicates()
                                 .rename(columns=new_col_names))
        else:
            end_stations_df = (main_df[[station_id, station_name]]
                               .drop_duplicates()
                               .rename(columns=new_col_names))
        
    # concactenate start and end stations names/ids
    all_stations_df = pd.concat([start_stations_df, end_stations_df], ignore_index=True)
    all_stations_df = all_stations_df.drop_duplicates(subset=['station_id', 'station_name'])
    
    # return df
    return all_stations_df

In [138]:
# get df of unique station pairs from citibike df
citibike_stations_df = combine_station_info(citibike_df)

# look at unique values
print(f'Number of rows: {len(citibike_stations_df)}')
citibike_stations_df.nunique()

Number of rows: 533


station_id      531
station_name    530
dtype: int64

In [181]:
# join citibike stations info with official stations_df
merged_stations_df = pd.merge(
    citibike_stations_df,
    stations_df,
    how='left',
    left_on='station_id',
    right_on='short_name'
)

# adding index label col for pk col
merged_stations_df['index_label'] = merged_stations_df.index

# match missing short_name values by station_name to name
missing_short_names = merged_stations_df[merged_stations_df.short_name.isna()].copy()

matching_short_names_df = pd.merge(
    missing_short_names[['station_id', 'station_name', 'index_label']],
    stations_df,
    how='left',
    left_on='station_name',
    right_on='name'
).rename(columns={
    'short_name': 'new_short_name',
    'region_id': 'new_region_id',
    'lat': 'new_lat',
    'lon': 'new_lon'
})

# combine them together again
merged_stations_df = pd.merge(
    merged_stations_df,
    matching_short_names_df[['index_label', 'new_short_name', 'new_region_id', 'new_lat', 'new_lon']],
    how='left',
    on='index_label'
)

# coalesce: if match found, use it
merged_stations_df['short_name'] = merged_stations_df['new_short_name'].combine_first(merged_stations_df['short_name'])
merged_stations_df['region_id'] = merged_stations_df['new_region_id'].combine_first(merged_stations_df['region_id'])
merged_stations_df['lat'] = merged_stations_df['new_lat'].combine_first(merged_stations_df['lat'])
merged_stations_df['lon'] = merged_stations_df['new_lon'].combine_first(merged_stations_df['lon'])

# drop old columns
merged_stations_df = merged_stations_df.drop(['new_short_name', 'new_region_id', 'new_lat', 'new_lon'], axis=1)

In [182]:
# look at unique value count
print(f'Number of rows: {len(merged_stations_df)}')
merged_stations_df.nunique()

Number of rows: 533


station_id      531
station_name    530
short_name      521
name            519
region_id         3
lat             520
lon             521
index_label     533
dtype: int64

### Separate unmatched station info

In [272]:
# save missing station info
missing_station_info_df = merged_stations_df[merged_stations_df['short_name'].isna() &
                                             merged_stations_df['station_name'].notna()][
                                                 ['station_id', 'station_name']].copy()

# # save station_info that isn't missing from official stations df
# official_merged_stations_df = merged_stations_df[merged_stations_df['short_name'].notna()][stations_df.columns.to_list()].copy()

In [193]:
# check unique values for missing station info
print(f'Number of rows: {len(missing_station_info_df)}')
missing_station_info_df.nunique()

Number of rows: 8


station_id      8
station_name    8
dtype: int64

### Fill in missing station info

In [250]:
# function to extract start stations with lat/lng
def extract_station_coords(df, side='start'):

    # make copy of df
    main_df = df.copy()

    # col names
    station_id = f'{side}_station_id'
    lat = f'{side}_lat'
    lng = f'{side}_lng'

    # extract station info with lat/lng
    station_coords_df = (
        main_df[[station_id, lat, lng]]
        .rename(columns= {
            station_id: 'station_id',
            lat: 'lat',
            lng: 'lon'
        })
    )

    # return
    return station_coords_df

##### fill in using averages of lat/lng --> accuracy ended up varying a decent amount

In [257]:
# extract station coordinates
start_station_coords_df = extract_station_coords(citibike_df, side='start')
end_station_coords_df = extract_station_coords(citibike_df, side='end')

# combine them
all_station_coords_df = pd.concat([start_station_coords_df, end_station_coords_df], ignore_index=True)

# group by station_id to get avg lat/lng
station_avg_coords_df = (
    all_station_coords_df.groupby('station_id')
    .agg({
        'lat': 'mean',
        'lon': 'mean'
    })
    .reset_index()
    .dropna(subset=['station_id'])  #drop nulls
)

# merge averages onto missing stations
filled_avg_missing_station_df = pd.merge(
    missing_station_info_df,
    station_avg_coords_df,
    how='left',
    on='station_id'
)

# display
filled_avg_missing_station_df

,station_id,station_name,lat,lon
0,HB503,Madison St & 10 St,40.749954,-74.035867
1,6839.10,Grand Army Plaza & Central Park S,40.747599,-74.013562
2,6801.01,E 59 St & Madison Ave,40.747641,-73.995481
3,5065.04,John St & William St,40.708621,-74.007222
4,6303.01,1 Ave & E 39 St,40.747140,-73.971130
5,5470.02,N Moore St & Hudson St,40.719961,-74.008443
6,6432.10,E 41 St & Madison Ave (SE corner),40.752049,-73.979635
7,6779.04,Broadway & W 51 St,40.762288,-73.983362


##### fill in using max count of duplicate lat/lng combos --> better accuracy

In [273]:
# extract station coordinates
start_station_coords_df = extract_station_coords(citibike_df, side='start')
end_station_coords_df = extract_station_coords(citibike_df, side='end')

# combine them
all_station_coords_df = pd.concat([start_station_coords_df, end_station_coords_df], ignore_index=True)

# filter for missing ids in new df and check for duplicate coordinate counts
missing_ids = missing_station_info_df['station_id'].tolist()
missing_coords_dupes_cnt = (all_station_coords_df[all_station_coords_df.station_id.isin(missing_ids)]
                            .groupby(['station_id','lat', 'lon'])
                            .size()
                            .reset_index(name='count'))

# get coordinates based on highest duplicate count (lat/lng combos that show up frequently)
max_index = missing_coords_dupes_cnt.groupby('station_id')['count'].idxmax()  #get index for each station's max cnt
best_missing_coords_df = missing_coords_dupes_cnt.loc[max_index].copy()

# merge back with missing stations_df to get station names
filled_missing_stations_df = pd.merge(
    missing_station_info_df,
    best_missing_coords_df[['station_id', 'lat', 'lon']],
    how='left',
    on='station_id'
).rename(columns={
    'station_id': 'short_name',
    'station_name': 'name'
})

# display
filled_missing_stations_df

,short_name,name,lat,lon
0,HB503,Madison St & 10 St,40.749943,-74.035865
1,6839.10,Grand Army Plaza & Central Park S,40.764397,-73.973715
2,6801.01,E 59 St & Madison Ave,40.763505,-73.971092
3,5065.04,John St & William St,40.708621,-74.007222
4,6303.01,1 Ave & E 39 St,40.747140,-73.971130
5,5470.02,N Moore St & Hudson St,40.719961,-74.008443
6,6432.10,E 41 St & Madison Ave (SE corner),40.752049,-73.979635
7,6779.04,Broadway & W 51 St,40.762288,-73.983362


### Combine station info

In [288]:
#updated_stations_df = pd.concat([stations_df[['short_name', 'name', 'lat', 'lon']], filled_missing_stations_df], ignore_index=True)
updated_stations_df = pd.concat([stations_df, filled_missing_stations_df], ignore_index=True)

# check unique values
print(f'Number of rows: {len(updated_stations_df)}')
print(updated_stations_df.nunique())

Number of rows: 2239
short_name    2239
name          2239
region_id        3
lat           2226
lon           2222
dtype: int64


### Check: see if all station ids in CitiBike df are in official station info table

In [325]:
# list missing station ids
updated_missing_station_ids = get_missing_station_ids(citibike_df, updated_stations_df)
updated_missing_station_ids

['6920.03', '6762.04', '5980.07', '6098.12', nan]

In [305]:
citibike_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,end_lng,member_casual,start_zip,end_zip,start_city,start_county,start_state,end_city,end_county,end_state
0,0744109F13385D1D,electric_bike,2024-01-15 15:18:07,2024-01-15 15:32:44,Morris Canal,JC072,Oakland Ave,JC022,40.712297,-74.038185,...,-74.052478,member,07302,07306,Jersey City,34017,NaN,Jersey City,34017,NaN
1,B1488BFEF9118000,classic_bike,2024-01-13 15:32:50,2024-01-13 15:36:18,JC Medical Center,JC110,Grove St PATH,JC115,40.715391,-74.049692,...,-74.043090,member,07302,07302,Jersey City,34017,NaN,Jersey City,34017,NaN
2,95A2FE8E51B4C836,classic_bike,2024-01-19 13:11:00,2024-01-19 13:14:44,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,...,-74.034344,member,07302,07302,Jersey City,34017,NaN,Jersey City,34017,NaN
3,95D9AFF6A1652DC1,classic_bike,2024-01-23 07:03:49,2024-01-23 07:07:11,Morris Canal,JC072,Exchange Pl,JC116,40.712419,-74.038526,...,-74.034344,member,07302,07302,Jersey City,34017,NaN,Jersey City,34017,NaN
4,5F7408988A83B1B3,classic_bike,2024-01-01 16:46:10,2024-01-01 16:50:31,Morris Canal,JC072,Harborside,JC104,40.712419,-74.038526,...,-74.034234,member,07302,07311,Jersey City,34017,NaN,Jersey City,34017,NaN


In [126]:
# list missing station ids
missing_station_ids = missing_station_ids(citibike_df)
missing_station_ids

['6920.03',
 '5065.04',
 '6801.01',
 '6762.04',
 '5980.07',
 '6432.10',
 '6839.10',
 '6779.04',
 '6098.12',
 '6303.01',
 '5470.02',
 'HB503',
 nan]